In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

### Cargar del dataset
url = 'https://raw.githubusercontent.com/lespinozahuiman/Datasets/main/DataSet_TF_v5.csv'

# data = pd.read_csv(url, index_col = 'InvoiceDate',parse_dates = ['InvoiceDate'])
data = pd.read_csv(url)
data.head()

# data.sort_values(by='InvoiceDate', ascending = False)

2024-05-08 03:59:33.671933: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Retailer,RetailerID,InvoiceDate,Region,State,City,Product,PricePerUnit,UnitsSold,TotalSales,OperatingProfit,SalesMethod
0,Walmart,1128299,17-06-2021,Southeast,Florida,Orlando,Women's Apparel,103.0,218.0,2245.0,1257.0,Online
1,West Gear,1128299,16-07-2021,South,Louisiana,New Orleans,Women's Apparel,103.0,163.0,1679.0,806.0,Online
2,Sports Direct,1197831,25-08-2021,South,Alabama,Birmingham,Men's Street Footwear,10.0,700.0,7000.0,3150.0,Outlet
3,Sports Direct,1197831,27-08-2021,South,Alabama,Birmingham,Women's Street Footwear,15.0,575.0,8625.0,3881.0,Outlet
4,Sports Direct,1197831,21-08-2021,South,Alabama,Birmingham,Women's Street Footwear,15.0,475.0,7125.0,3206.0,Outlet


In [2]:
# Convertir la columna InvoiceDate a datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Creando las variables mes y año
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year

data.columns

Columns_drop = ['RetailerID','InvoiceDate','OperatingProfit']

# Eliminar las columnas especificadas
data = data.drop(columns=Columns_drop)

data_ordenada = ['Year','Month','Retailer','Region','State','City','Product']

data

# # Variables categóricas para aplicar One-Hot Encoding
# categorical_cols = ['Retailer', 'Region', 'State', 'City', 'Product']

# # Aplicar One-Hot Encoding
# encoded_data = pd.get_dummies(data, columns=categorical_cols)

# encoded_data

/var/folders/m2/030rdksn1yg5r7l01hsf80bh0000gn/T/ipykernel_20609/2550408268.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])


,Retailer,Region,State,City,Product,PricePerUnit,UnitsSold,TotalSales,SalesMethod,Month,Year
0,Walmart,Southeast,Florida,Orlando,Women's Apparel,103.0,218.0,2245.0,Online,6,2021
1,West Gear,South,Louisiana,New Orleans,Women's Apparel,103.0,163.0,1679.0,Online,7,2021
2,Sports Direct,South,Alabama,Birmingham,Men's Street Footwear,10.0,700.0,7000.0,Outlet,8,2021
3,Sports Direct,South,Alabama,Birmingham,Women's Street Footwear,15.0,575.0,8625.0,Outlet,8,2021
4,Sports Direct,South,Alabama,Birmingham,Women's Street Footwear,15.0,475.0,7125.0,Outlet,8,2021
...,...,...,...,...,...,...,...,...,...,...,...
9636,Amazon,Northeast,Maine,Portland,Men's Apparel,65.0,150.0,9750.0,Outlet,11,2021
9637,Amazon,West,Alaska,Anchorage,Women's Athletic Footwear,65.0,150.0,9750.0,Outlet,10,2021
9638,Amazon,West,Alaska,Anchorage,Men's Apparel,75.0,150.0,11250.0,Outlet,10,2021
9639,West Gear,Midwest,Indiana,Indianapolis,Men's Apparel,65.0,150.0,9750.0,In-store,10,2021


In [3]:
### Convertir la columna InvoiceDate a datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

### Creando las variables mes y año
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year

x = ['Year','Month','Retailer','Region','State','City','Product']

# ### Agrupando por meses

result = data.groupby(x).agg({'TotalSales': 'sum', 'PricePerUnit': 'mean'}).reset_index()

result

# result1 = data.groupby(x).apply(lambda a: pd.Series({
#     'Cantidad_Total_Vendida': a['UnitsSold'].sum(),
#     'Precio_Promedio': a['PricePerUnit'].mean(),
#     'Total_Ventas_Real': a['TotalSales'].sum()
# }))




KeyError: 'InvoiceDate'

In [4]:

data = pd.read_csv(url)  # Asegúrate de reemplazar 'datos.csv' con el nombre de tu archivo CSV

# Convertir la columna de fecha a formato datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Obtener la fecha del último registro en el conjunto de datos
ultima_fecha = data['InvoiceDate'].max()
print("ultima_fecha : ",ultima_fecha)

# Calcular la fecha del primer día del mes anterior
primer_dia_mes_anterior = ultima_fecha.replace(day=1) - pd.DateOffset(months=1)
print("primer_dia_mes_anterior : ", primer_dia_mes_anterior)

# Calcular la fecha del último día del mes anterior
ultimo_dia_mes_anterior = primer_dia_mes_anterior + pd.offsets.MonthEnd()
print("ultimo_dia_mes_anterior : ", ultimo_dia_mes_anterior)

# Filtrar las filas correspondientes al último mes
ventas_ultimo_mes = pd.DataFrame(data[(data['InvoiceDate'] >= primer_dia_mes_anterior) & (data['InvoiceDate'] <= ultimo_dia_mes_anterior)])

# Calcular el TotalSales del último mes
total_ventas_ultimo_mes = ventas_ultimo_mes['TotalSales'].sum()

print("El TotalSales del último mes es:", total_ventas_ultimo_mes)

ventas_ultimo_mes.sort_values(by='InvoiceDate', ascending = False)


ultima_fecha :  2021-12-31 00:00:00
primer_dia_mes_anterior :  2021-11-01 00:00:00
ultimo_dia_mes_anterior :  2021-11-30 00:00:00
El TotalSales del último mes es: 6396121.0


/var/folders/m2/030rdksn1yg5r7l01hsf80bh0000gn/T/ipykernel_20609/2756714693.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])


,Retailer,RetailerID,InvoiceDate,Region,State,City,Product,PricePerUnit,UnitsSold,TotalSales,OperatingProfit,SalesMethod
5825,Foot Locker,1185732,2021-11-30,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,39.0,56.0,218.0,103.0,Outlet
9601,Foot Locker,1185732,2021-11-30,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,55.0,175.0,9625.0,3850.0,Outlet
6130,Foot Locker,1185732,2021-11-30,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,51.0,44.0,224.0,119.0,Online
9439,Foot Locker,1185732,2021-11-29,Northeast,Pennsylvania,Philadelphia,Women's Street Footwear,55.0,225.0,12375.0,4950.0,Outlet
5714,Foot Locker,1185732,2021-11-29,Northeast,Pennsylvania,Philadelphia,Women's Street Footwear,51.0,59.0,301.0,150.0,Online
...,...,...,...,...,...,...,...,...,...,...,...,...
3002,West Gear,1185732,2021-11-01,Midwest,Missouri,St. Louis,Women's Apparel,24.0,138.0,331.0,212.0,Online
1377,West Gear,1185732,2021-11-01,Midwest,Missouri,St. Louis,Men's Street Footwear,23.0,196.0,451.0,248.0,Online
3162,West Gear,1185732,2021-11-01,Midwest,Missouri,St. Louis,Men's Athletic Footwear,23.0,133.0,306.0,144.0,Online
3293,Kohl's,1189833,2021-11-01,Midwest,Montana,Billings,Women's Street Footwear,42.0,128.0,538.0,306.0,Online


In [5]:
# Suponiendo que 'data' es tu DataFrame y 'InvoiceDate' es la columna de fecha
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Ordenar el DataFrame por fecha en orden descendente
data_sorted = data.sort_values(by='InvoiceDate', ascending=False)

# Obtener la fecha máxima en el DataFrame
fecha_maxima = data['InvoiceDate'].max()

# Calcular la fecha de hace 3 meses
fecha_hace_3_meses = fecha_maxima - timedelta(days=3*30)  # Suponiendo que un mes tiene 30 días

# Filtrar las filas correspondientes a los últimos 3 meses
ultimos_3_meses = data_sorted[data_sorted['InvoiceDate'] >= fecha_hace_3_meses]

ultimos_3_meses

,Retailer,RetailerID,InvoiceDate,Region,State,City,Product,PricePerUnit,UnitsSold,TotalSales,OperatingProfit,SalesMethod
6312,Amazon,1185732,2021-12-31,Northeast,Maine,Portland,Men's Apparel,52.0,36.0,187.0,69.0,Online
6041,Foot Locker,1185732,2021-12-31,Northeast,Pennsylvania,Philadelphia,Men's Apparel,63.0,47.0,296.0,136.0,Online
9605,Foot Locker,1185732,2021-12-31,Northeast,Pennsylvania,Philadelphia,Men's Apparel,70.0,175.0,12250.0,4288.0,Outlet
5826,Foot Locker,1185732,2021-12-31,Northeast,Pennsylvania,Philadelphia,Men's Apparel,46.0,56.0,258.0,100.0,Outlet
3413,Amazon,1185732,2021-12-31,Northeast,Maine,Portland,Men's Apparel,55.0,125.0,6875.0,1719.0,Outlet
...,...,...,...,...,...,...,...,...,...,...,...,...
2500,Foot Locker,1128299,2021-10-02,West,Hawaii,Honolulu,Men's Apparel,50.0,150.0,7500.0,1125.0,Outlet
1302,West Gear,1185732,2021-10-02,Midwest,Indiana,Indianapolis,Men's Apparel,45.0,200.0,9000.0,2700.0,In-store
5927,West Gear,1185732,2021-10-02,Midwest,Wisconsin,Milwaukee,Men's Athletic Footwear,34.0,52.0,177.0,90.0,Online
3270,Kohl's,1189833,2021-10-02,Midwest,Montana,Billings,Women's Street Footwear,49.0,130.0,637.0,376.0,Online
